In [1]:
# add path where .py files can be imported
import sys
sys.path.insert(0, './IndustrialSmokePlumeDetection/segmentation')

In [2]:
# u-net training process
import numpy as np
import random
import torch
from torch import nn, optim
from tqdm.autonotebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import argparse
from sklearn.metrics import jaccard_score

from network_ori import *
from data import create_dataset

import warnings
warnings.filterwarnings('ignore')

#initialize model and device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = R2AttU_Net(img_ch=12, output_ch=1)
model.to(device)

print('running on...', device)


#default parameters
md = 'r2attunet'
ep = 300 # Number of epochs
bs = 60 # Batch size
lr = 0.5 # Learning rate
mo = 0.7 # Momentum


# setup tensorboard writer
writer = SummaryWriter('../runs/'+"{}_ep{:0d}_lr{:.0e}_bs{:03d}_mo{:.1f}/".format(
    md, ep, lr, bs, mo))

# initialize loss function
loss = nn.BCEWithLogitsLoss()

# initialize optimizer
opt = optim.SGD(model.parameters(), lr=lr, momentum=mo)

# initialize scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min',
                                                 factor=0.5, threshold=1e-4,
                                                 min_lr=1e-6)



# prepare training and validation data loaders
data_all = create_dataset(
    datadir = '../data', # /path/to/image/data/
    seglabeldir = '../labels', # /path/to/segmentation/labels/for/training/
    mult=1)

len_all = len(data_all)
split_1 = (15*len_all) // 100
split_2 = 2*split_1
indices = list(range(len_all))
random.seed(9001)
random.shuffle(indices)

train_sampler = SubsetRandomSampler(indices[split_2:])
val_sampler = SubsetRandomSampler(indices[split_1:split_2])
test_sampler = SubsetRandomSampler(indices[:split_1])

train_dl = DataLoader(data_all, batch_size=bs, num_workers=6, 
                      pin_memory=True, sampler=train_sampler)
val_dl = DataLoader(data_all, batch_size=bs, num_workers=6, 
                      pin_memory=True, sampler=val_sampler)


# run training
for epoch in range(ep):

    model.train()

    train_loss_total = 0
    train_ious = []
    train_acc_total = 0
    train_arearatios = []
    progress = tqdm(enumerate(train_dl), desc="Train Loss: ",
                    total=len(train_dl))
    for i, batch in progress:
        x = batch['img'].float().to(device)
        y = batch['fpt'].float().to(device)

        output = model(x)

        # derive binary segmentation map from prediction
        output_binary = np.zeros(output.shape)
        output_binary[output.cpu().detach().numpy() >= 0] = 1

        # derive IoU values
        ious = []
        for j in range(y.shape[0]):
            z = jaccard_score(y[j].flatten().cpu().detach().numpy(),
                        output_binary[j][0].flatten())
            if (np.sum(output_binary[j][0]) != 0 and
                np.sum(y[j].cpu().detach().numpy()) != 0):
                train_ious.append(z)

        # derive scalar binary labels on a per-image basis
        y_bin = np.array(np.sum(y.cpu().detach().numpy(),
                                axis=(1,2)) != 0).astype(int)
        pred_bin = np.array(np.sum(output_binary,
                                    axis=(1,2,3)) != 0).astype(int)

        # derive image-wise accuracy for this batch
        train_acc_total += accuracy_score(y_bin, pred_bin)

        # derive loss
        loss_epoch = loss(output, y.unsqueeze(dim=1))
        train_loss_total += loss_epoch.item()
        progress.set_description("Train Loss: {:.4f}".format(
            train_loss_total/(i+1)))

        # derive smoke areas
        area_pred = np.sum(output_binary, axis=(1,2,3))
        area_true = np.sum(y.cpu().detach().numpy(), axis=(1,2))

        # derive smoke area ratios
        arearatios = []
        for k in range(len(area_pred)):
            if area_pred[k] == 0 and area_true[k] == 0:
                arearatios.append(1)
            elif area_true[k] == 0:
                arearatios.append(0)
            else:
                arearatios.append(area_pred[k]/area_true[k])
        train_arearatios = np.ravel([*train_arearatios, *arearatios])

        # learning
        opt.zero_grad()
        loss_epoch.backward()
        opt.step()

    # logging
    writer.add_scalar("training loss", train_loss_total/(i+1), epoch)
    writer.add_scalar("training iou", np.average(train_ious), epoch)
    writer.add_scalar("training acc", train_acc_total/(i+1), epoch)
    writer.add_scalar('training arearatio mean',
                        np.average(train_arearatios), epoch)
    writer.add_scalar('training arearatio std',
                        np.std(train_arearatios), epoch)
    writer.add_scalar('learning_rate', opt.param_groups[0]['lr'], epoch)

    torch.cuda.empty_cache()

    # evaluation
    model.eval()
    val_loss_total = 0
    val_ious = []
    val_acc_total = 0
    val_arearatios = []
    progress = tqdm(enumerate(val_dl), desc="val Loss: ",
                    total=len(val_dl))
    for j, batch in progress:
        x = batch['img'].float().to(device)
        y = batch['fpt'].float().to(device)

        output = model(x)

        # derive loss
        loss_epoch = loss(output, y.unsqueeze(dim=1))
        val_loss_total += loss_epoch.item()

        # derive binary segmentation map from prediction
        output_binary = np.zeros(output.shape)
        output_binary[output.cpu().detach().numpy() >= 0] = 1

        # derive IoU values
        ious = []
        for k in range(y.shape[0]):
            z = jaccard_score(y[k].flatten().cpu().detach().numpy(),
                        output_binary[k][0].flatten())
            if (np.sum(output_binary[k][0]) != 0 and 
                np.sum(y[k].cpu().detach().numpy()) != 0):
                val_ious.append(z)

        # derive scalar binary labels on a per-image basis
        y_bin = np.array(np.sum(y.cpu().detach().numpy(),
                                axis=(1,2)) != 0).astype(int)
        pred_bin = np.array(np.sum(output_binary,
                                    axis=(1,2,3)) != 0).astype(int)

        # derive image-wise accuracy for this batch
        val_acc_total += accuracy_score(y_bin, pred_bin)

        # derive smoke areas
        area_pred = np.sum(output_binary, axis=(1,2,3))
        area_true = np.sum(y.cpu().detach().numpy(), axis=(1,2))

        # derive smoke area ratios
        arearatios = []
        for k in range(len(area_pred)):
            if area_pred[k] == 0 and area_true[k] == 0:
                arearatios.append(1)
            elif area_true[k] == 0:
                arearatios.append(0)
            else:
                arearatios.append(area_pred[k]/area_true[k])
        val_arearatios = np.ravel([*val_arearatios, *arearatios])
        
        progress.set_description("val Loss: {:.4f}".format(
            val_loss_total/(j+1)))

    # logging
    writer.add_scalar("val loss", val_loss_total/(j+1), epoch)
    writer.add_scalar("val iou", np.average(val_ious), epoch)
    writer.add_scalar("val acc", val_acc_total/(j+1), epoch)
    writer.add_scalar('val arearatio mean',
                        np.average(val_arearatios), epoch)
    writer.add_scalar('val arearatio std',
                        np.std(val_arearatios), epoch)
    
    print(("Epoch {:d}: train loss={:.3f}, val loss={:.3f}, "
            "train iou={:.3f}, val iou={:.3f}, "
            "train acc={:.3f}, val acc={:.3f}").format(
                epoch+1, train_loss_total/(i+1), val_loss_total/(j+1),
                np.average(train_ious), np.average(val_ious),
                train_acc_total/(i+1), val_acc_total/(j+1)))
    
    # save model checkpoint
    if epoch % 1 == 0:
        torch.save(model.state_dict(),
        '../models/{}_ep{:0d}_lr{:.0e}_bs{:02d}_mo{:.1f}_{:03d}.model'.format(
            md, ep, lr, bs, mo, epoch))

    writer.flush()
    scheduler.step(val_loss_total/(j+1))
    torch.cuda.empty_cache()


writer.close()

running on... cuda:0


Train Loss:   0%|          | 0/34 [00:00<?, ?it/s]

val Loss:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1: train loss=0.119, val loss=0.079, train iou=0.201, val iou=0.279, train acc=0.643, val acc=0.685


Train Loss:   0%|          | 0/34 [00:00<?, ?it/s]

...

Train Loss:   0%|          | 0/34 [00:00<?, ?it/s]

val Loss:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 110: train loss=0.023, val loss=0.071, train iou=0.630, val iou=0.442, train acc=0.944, val acc=0.869


Train Loss:   0%|          | 0/34 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#deeplabv3+ resnet101
import sys
sys.path.insert(0,'./IndustrialSmokePlumeDetection/segmentation')
sys.path.insert(0,'./pytorch-deeplab-xception')

import numpy as np
import random
import torch
from torch import nn, optim
from tqdm.autonotebook import tqdm
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import argparse
from sklearn.metrics import jaccard_score

from modeling.deeplab import *
from data import create_dataset

import warnings
warnings.filterwarnings('ignore')



#initialize model and device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = DeepLab(num_classes=1, backbone='resnet')
model.to(device)

print('running on...', device)


#default parameters
md = 'dlabv3res'
ep = 300 # Number of epochs
bs = 100 # Batch size
lr = 0.1 # Learning rate
mo = 0.7 # Momentum


# setup tensorboard writer
writer = SummaryWriter('../runs/'+"{}_ep{:0d}_lr{:.0e}_bs{:03d}_mo{:.1f}/".format(
    md, ep, lr, bs, mo))

# initialize loss function
loss = nn.BCEWithLogitsLoss()

# initialize optimizer
opt = optim.SGD(model.parameters(), lr=lr, momentum=mo)

# initialize scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, 'min',
                                                 factor=0.5, threshold=1e-4,
                                                 min_lr=1e-6)



# prepare training and validation data loaders
data_all = create_dataset(
    datadir = '../data', # /path/to/image/data/
    seglabeldir = '../labels', # /path/to/segmentation/labels/for/training/
    mult=1)

len_all = len(data_all)
split_1 = (15*len_all) // 100
split_2 = 2*split_1
indices = list(range(len_all))
random.seed(9001)
random.shuffle(indices)

train_sampler = SubsetRandomSampler(indices[split_2:])
val_sampler = SubsetRandomSampler(indices[split_1:split_2])
test_sampler = SubsetRandomSampler(indices[:split_1])

train_dl = DataLoader(data_all, batch_size=bs, num_workers=6, 
                      pin_memory=True, sampler=train_sampler)
val_dl = DataLoader(data_all, batch_size=bs, num_workers=6, 
                      pin_memory=True, sampler=val_sampler)


# run training
for epoch in range(ep):

    model.train()

    train_loss_total = 0
    train_ious = []
    train_acc_total = 0
    train_arearatios = []
    progress = tqdm(enumerate(train_dl), desc="Train Loss: ",
                    total=len(train_dl))
    for i, batch in progress:
        x = batch['img'].float().to(device)
        y = batch['fpt'].float().to(device)

        output = model(x)

        # derive binary segmentation map from prediction
        output_binary = np.zeros(output.shape)
        output_binary[output.cpu().detach().numpy() >= 0] = 1

        # derive IoU values
        ious = []
        for j in range(y.shape[0]):
            z = jaccard_score(y[j].flatten().cpu().detach().numpy(),
                        output_binary[j][0].flatten())
            if (np.sum(output_binary[j][0]) != 0 and
                np.sum(y[j].cpu().detach().numpy()) != 0):
                train_ious.append(z)

        # derive scalar binary labels on a per-image basis
        y_bin = np.array(np.sum(y.cpu().detach().numpy(),
                                axis=(1,2)) != 0).astype(int)
        pred_bin = np.array(np.sum(output_binary,
                                    axis=(1,2,3)) != 0).astype(int)

        # derive image-wise accuracy for this batch
        train_acc_total += accuracy_score(y_bin, pred_bin)

        # derive loss
        loss_epoch = loss(output, y.unsqueeze(dim=1))
        train_loss_total += loss_epoch.item()
        progress.set_description("Train Loss: {:.4f}".format(
            train_loss_total/(i+1)))

        # derive smoke areas
        area_pred = np.sum(output_binary, axis=(1,2,3))
        area_true = np.sum(y.cpu().detach().numpy(), axis=(1,2))

        # derive smoke area ratios
        arearatios = []
        for k in range(len(area_pred)):
            if area_pred[k] == 0 and area_true[k] == 0:
                arearatios.append(1)
            elif area_true[k] == 0:
                arearatios.append(0)
            else:
                arearatios.append(area_pred[k]/area_true[k])
        train_arearatios = np.ravel([*train_arearatios, *arearatios])

        # learning
        opt.zero_grad()
        loss_epoch.backward()
        opt.step()

    # logging
    writer.add_scalar("training loss", train_loss_total/(i+1), epoch)
    writer.add_scalar("training iou", np.average(train_ious), epoch)
    writer.add_scalar("training acc", train_acc_total/(i+1), epoch)
    writer.add_scalar('training arearatio mean',
                        np.average(train_arearatios), epoch)
    writer.add_scalar('training arearatio std',
                        np.std(train_arearatios), epoch)
    writer.add_scalar('learning_rate', opt.param_groups[0]['lr'], epoch)

    torch.cuda.empty_cache()

    # evaluation
    model.eval()
    val_loss_total = 0
    val_ious = []
    val_acc_total = 0
    val_arearatios = []
    progress = tqdm(enumerate(val_dl), desc="val Loss: ",
                    total=len(val_dl))
    for j, batch in progress:
        x = batch['img'].float().to(device)
        y = batch['fpt'].float().to(device)

        output = model(x)

        # derive loss
        loss_epoch = loss(output, y.unsqueeze(dim=1))
        val_loss_total += loss_epoch.item()

        # derive binary segmentation map from prediction
        output_binary = np.zeros(output.shape)
        output_binary[output.cpu().detach().numpy() >= 0] = 1

        # derive IoU values
        ious = []
        for k in range(y.shape[0]):
            z = jaccard_score(y[k].flatten().cpu().detach().numpy(),
                        output_binary[k][0].flatten())
            if (np.sum(output_binary[k][0]) != 0 and 
                np.sum(y[k].cpu().detach().numpy()) != 0):
                val_ious.append(z)

        # derive scalar binary labels on a per-image basis
        y_bin = np.array(np.sum(y.cpu().detach().numpy(),
                                axis=(1,2)) != 0).astype(int)
        pred_bin = np.array(np.sum(output_binary,
                                    axis=(1,2,3)) != 0).astype(int)

        # derive image-wise accuracy for this batch
        val_acc_total += accuracy_score(y_bin, pred_bin)

        # derive smoke areas
        area_pred = np.sum(output_binary, axis=(1,2,3))
        area_true = np.sum(y.cpu().detach().numpy(), axis=(1,2))

        # derive smoke area ratios
        arearatios = []
        for k in range(len(area_pred)):
            if area_pred[k] == 0 and area_true[k] == 0:
                arearatios.append(1)
            elif area_true[k] == 0:
                arearatios.append(0)
            else:
                arearatios.append(area_pred[k]/area_true[k])
        val_arearatios = np.ravel([*val_arearatios, *arearatios])
        
        progress.set_description("val Loss: {:.4f}".format(
            val_loss_total/(j+1)))

    # logging
    writer.add_scalar("val loss", val_loss_total/(j+1), epoch)
    writer.add_scalar("val iou", np.average(val_ious), epoch)
    writer.add_scalar("val acc", val_acc_total/(j+1), epoch)
    writer.add_scalar('val arearatio mean',
                        np.average(val_arearatios), epoch)
    writer.add_scalar('val arearatio std',
                        np.std(val_arearatios), epoch)
    
    print(("Epoch {:d}: train loss={:.3f}, val loss={:.3f}, "
            "train iou={:.3f}, val iou={:.3f}, "
            "train acc={:.3f}, val acc={:.3f}").format(
                epoch+1, train_loss_total/(i+1), val_loss_total/(j+1),
                np.average(train_ious), np.average(val_ious),
                train_acc_total/(i+1), val_acc_total/(j+1)))
    
    # save model checkpoint
    if epoch % 1 == 0:
        torch.save(model.state_dict(),
        '../models/{}_ep{:0d}_lr{:.0e}_bs{:02d}_mo{:.1f}_{:03d}.model'.format(
            md, ep, lr, bs, mo, epoch))

    writer.flush()
    scheduler.step(val_loss_total/(j+1))
    torch.cuda.empty_cache()


writer.close()

running on... cuda:0


Train Loss:   0%|          | 0/20 [00:00<?, ?it/s]